# Data loading and preprocessing, utility function definition

## Imports and data loading

In [ ]:
import gc
import itertools
import time

import numpy as np
import pandas as pd
import seaborn as sns
from IPython.core.debugger import set_trace
from pandas.tseries.offsets import MonthEnd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import lightgbm as lgbm
# A few functions are imported from a utility script, have a look at it if you like.
import futuresalesutility as fu

Load the data and convert the date column in the training data to the datetime dtype to enable datetime operations.

In [ ]:
item_categories_extra = pd.read_csv('../input/predict-future-sales-extra/item_categories_enhanced.csv')
item_categories_extra = item_categories_extra.drop(columns=['category_name', 'supercategory', 'platform'])
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")

## Data cleaning

* Correct duplicate shop names
* Drop a few duplicate items in the training set
* Drop shops which are not in the test set (these are only a few of these and they tend to be strange in some way, e.g. low sales, not operating for long)+
* Drop categories 8 and 80 as these are for tickets to an annual exhibition which are not sold in the test month
* Remove outliers and negative values for the item_cnt_day and item_price features (these are few in number but cause problems when generating some features)

In [ ]:
# Correct shop labels
train.loc[train.shop_id == 0, "shop_id"] = 57
train.loc[train.shop_id == 1, "shop_id"] = 58
train.loc[train.shop_id == 11, "shop_id"] = 10
# Drop shops not in test set
testshops = test.shop_id.unique()
train = train.loc[train["shop_id"].isin(testshops), :]
del testshops
# Drop duplicates
train = train.drop_duplicates()
# Drop categories 8 and 80
train = train.merge(items[["item_id", "item_category_id"]], on="item_id", how="left")
train = train[~train.item_category_id.isin([8, 80])]
# Clip outliers and  remove items with a negative sales price or item_cnt_day
train = train.query("(item_price>0) & (item_cnt_day>0)")
train.loc[:, "item_price"] = train.loc[:, "item_price"].clip(0, train["item_price"].quantile(0.9999))
train.loc[:, "item_cnt_day"] = train.loc[:, "item_cnt_day"].clip(0, train["item_cnt_day"].quantile(0.999))

## Preprocessing

Create a training matrix similar to the test items by aggregating the sales to the month level and creating items for every possible combination of shops and items featured in each individual month of the training data. Additionally, concatenate test to train data to enable creation of features for the test items.

In [ ]:
def create_testlike_train(sales_train, test=None):
    # Create a date_block_num / item_id / shop_id index using all combinations of item_id and shop_id occurring within each date_block
    # Optionally concatenate the test items to the end
    indexlist = []
    for i in sales_train.date_block_num.unique():
        x = itertools.product(
            [i],
            sales_train.loc[sales_train.date_block_num == i].shop_id.unique(),
            sales_train.loc[sales_train.date_block_num == i].item_id.unique(),
        )
        indexlist.append(np.array(list(x)))
    df = pd.DataFrame(
        data=np.concatenate(indexlist, axis=0), columns=["date_block_num", "shop_id", "item_id"],
    )

    # Add revenue column to sales_train
    sales_train["item_revenue_day"] = sales_train["item_price"] * sales_train["item_cnt_day"]
    # Aggregate item_id / shop_id item_cnts and revenue at the month level
    sales_train_grouped = sales_train.groupby(["date_block_num", "shop_id", "item_id"]).agg(
        item_cnt_month=pd.NamedAgg(column="item_cnt_day", aggfunc="sum"),
        item_revenue_month=pd.NamedAgg(column="item_revenue_day", aggfunc="sum"),
    )

    # Merge the grouped data with the index
    df = df.merge(sales_train_grouped, how="left", on=["date_block_num", "shop_id", "item_id"],)

    if test is not None:
        test["date_block_num"] = 34
        test["date_block_num"] = test["date_block_num"].astype(np.int8)
        test["shop_id"] = test.shop_id.astype(np.int8)
        test["item_id"] = test.item_id.astype(np.int16)
        test = test.drop(columns="ID")

        df = pd.concat([df, test[["date_block_num", "shop_id", "item_id"]]])

    # Fill empty item_cnt entries with 0
    df.item_cnt_month = df.item_cnt_month.fillna(0)
    df.item_revenue_month = df.item_revenue_month.fillna(0)

    return df

In [ ]:
matrix = create_testlike_train(train, test)
matrix = fu.reduce_mem_usage(matrix, silent=False)
oldcols = matrix.columns

# Feature engineering  
In this section predictor feature columns are generated and added to the matrix

## Time features
Time features such as item and shop age. Includes some features with day resolutions, such as the number of days since the last sale of the same item, and average sales per day in the previous month, taking into account items or shops which were not available for the whole month (items are assumed to be available from the the date of their first sale and are assumed to be released in all shops on the same date).  

Other features are the date of the first item sale anywhere and in the same shop, the time since the first and last sales anywhere and in the same shop, and the length of the current month. The datetime dtype is used for calculations.

In [ ]:
def add_time_features(m, train, correct_item_cnt_day=False):
    from pandas.tseries.offsets import Day, MonthBegin, MonthEnd

    def item_shop_age_months(m):
        m["item_age"] = m.groupby("item_id")["date_block_num"].transform(
            lambda x: x - x.min()
        )
        m["new_item"] = m["item_age"] == 0
        m["new_item"] = m["new_item"].astype("int8")
        m["shop_age"] = (
            m.groupby("shop_id")["date_block_num"]
            .transform(lambda x: x - x.min())
            .astype("int8")
        )
        m["new_shop"] = m.shop_age == 0
        m["new_shop"] = m["new_shop"].astype("int8")
        return m

    m = item_shop_age_months(m)

    # Add dummy values for the test month so that features are created correctly
    dummies = m.loc[m.date_block_num == 34, ["date_block_num", "shop_id", "item_id"]]
    dummies = dummies.assign(
        date=pd.to_datetime("2015-11-30"),
        item_price=1,
        item_cnt_day=0,
        item_revenue_day=0,
    )
    train = pd.concat([train, dummies])
    del dummies

    month_last_day = train.groupby("date_block_num").date.max().rename("month_last_day")
    month_last_day[~month_last_day.dt.is_month_end] = (
        month_last_day[~month_last_day.dt.is_month_end] + MonthEnd()
    )
    month_first_day = train.groupby("date_block_num").date.min().rename("month_first_day")
    month_first_day[~month_first_day.dt.is_month_start] = (
        month_first_day[~month_first_day.dt.is_month_start] - MonthBegin()
    )
    month_length = (month_last_day - month_first_day + Day()).rename("month_length")
    first_shop_date = train.groupby("shop_id").date.min().rename("first_shop_date")
    first_item_date = train.groupby("item_id").date.min().rename("first_item_date")
    first_shop_item_date = (
        train.groupby(["shop_id", "item_id"]).date.min().rename("first_shop_item_date")
    )

    m = m.merge(month_first_day, left_on="date_block_num", right_index=True, how="left")
    m = m.merge(month_last_day, left_on="date_block_num", right_index=True, how="left")
    m = m.merge(month_length, left_on="date_block_num", right_index=True, how="left")
    m = m.merge(first_shop_date, left_on="shop_id", right_index=True, how="left")
    m = m.merge(first_item_date, left_on="item_id", right_index=True, how="left")
    m = m.merge(
        first_shop_item_date, left_on=["shop_id", "item_id"], right_index=True, how="left"
    )

    # Calculate how long the item was sold for in each month and use this to calculate average sales per day
    m["shop_open_days"] = m["month_last_day"] - m["first_shop_date"] + Day()
    m["item_first_sale_days"] = m["month_last_day"] - m["first_item_date"] + Day()
    m["item_in_shop_days"] = (
        m[["shop_open_days", "item_first_sale_days", "month_length"]].min(axis=1).dt.days
    )
    m["item_cnt_day_avg"] = m["item_cnt_month"] / m["item_in_shop_days"]
    m["month_length"] = m["month_length"].dt.days

    # Calculate the time differences from the beginning of the month so they can be used as features without lagging
    m["shop_open_days"] = m["month_first_day"] - m["first_shop_date"]
    m["item_first_sale_days"] = m["month_first_day"] - m["first_item_date"]
    m["shop_item_first_sale_days"] = m["month_first_day"] - m["first_shop_item_date"]
    m["shop_open_days"] = m["shop_open_days"].dt.days.fillna(0).clip(lower=0)
    m["item_first_sale_days"] = m["item_first_sale_days"].dt.days.fillna(0).clip(lower=0)
    m["shop_item_first_sale_days"] = (
        m["shop_item_first_sale_days"].dt.days.fillna(0).clip(lower=0)
    )

    # Change the first sale date feature to integer format indexed from the first day in the training data
    m["first_day"] = train.date.min()
    m["first_item_date"] = (m["first_item_date"] - m["first_day"]).dt.days
    m["first_shop_item_date"] = (m["first_shop_item_date"] - m["first_day"]).dt.days

    # Add days since last sale
    def last_sale_days(matrix):
        last_shop_item_dates = []
        last_item_dates = []
        for dbn in tqdm(range(1, 35)):
            lsid_temp = (
                train.query(f"date_block_num<{dbn}")
                .groupby(["shop_id", "item_id"])
                .date.max()
                .rename("last_shop_item_sale_date")
                .reset_index()
            )
            lid_temp = (
                lsid_temp.groupby("item_id")
                .last_shop_item_sale_date.max()
                .rename("last_item_sale_date")
                .reset_index()
            )
            lsid_temp["date_block_num"] = dbn
            lid_temp["date_block_num"] = dbn
            last_shop_item_dates.append(lsid_temp)
            last_item_dates.append(lid_temp)
        last_shop_item_dates = pd.concat(last_shop_item_dates)
        last_item_dates = pd.concat(last_item_dates)
        matrix = matrix.merge(
            last_shop_item_dates, on=["date_block_num", "shop_id", "item_id"], how="left"
        )
        matrix = matrix.merge(
            last_item_dates, on=["date_block_num", "item_id"], how="left"
        )

        def days_since_last_feat(m, feat_name, date_feat_name, missingval):
            m[feat_name] = (m["month_first_day"] - m[date_feat_name]).dt.days
            m.loc[m[feat_name] > 2000, feat_name] = missingval
            m.loc[m[feat_name].isna(), feat_name] = missingval
            return m

        matrix = days_since_last_feat(
            matrix, "last_shop_item_sale_days", "last_shop_item_sale_date", 9999
        )
        matrix = days_since_last_feat(
            matrix, "last_item_sale_days", "last_item_sale_date", 9999
        )
        matrix = matrix.drop(columns=["last_shop_item_sale_date", "last_item_sale_date"])
        return matrix

    m = last_sale_days(m)
    # Month
    m["month"] = m["month_first_day"].dt.month

    m = m.drop(
        columns=[
            "first_day",
            "month_first_day",
            "month_last_day",
            "first_shop_date",
            "item_in_shop_days",
            "first_shop_item_date",
        ]
    )

    if correct_item_cnt_day == True:
        m["item_cnt_month_original"] = m["item_cnt_month"]
        m["item_cnt_month"] = m["item_cnt_day_avg"] * m["month_length"]

    return m

In [ ]:
matrix = add_time_features(matrix, train, False)
print("Time features created")

Additionally clip the corrected monthly item counts to remove the effect of outliers, but save the unclipped version also.

In [ ]:
matrix['item_cnt_month_unclipped'] = matrix.loc[:,'item_cnt_month']
matrix['item_cnt_month'] = matrix['item_cnt_month'].clip(lower=0, upper=20)

## Price features  

Last mean price for a month in which the item was sold, the difference of the last price from the historical mean (calculated with an expanding window), the difference of the last mean price from the first sale price in the data (proxy for release price), and the difference of the last mean price from the mean for the category in the same month.  

In [ ]:
def add_price_features(matrix, train):
    # Get mean prices per month from train dataframe
    price_features = train.groupby(["date_block_num", "item_id"]).item_price.mean()
    price_features = pd.DataFrame(price_features)
    price_features = price_features.reset_index()
    # Calculate expanding mean and join to dataframe
    emp = price_features.groupby("item_id").item_price.expanding().mean()
    emp = emp.reset_index(level=0)
    price_features["exp_mean_price"] = emp["item_price"]
    del emp
    # Get first sale price for item
    price_features["start_price"] = price_features.groupby(
        "item_id"
    ).item_price.transform(lambda x: x.to_numpy()[0])
    # Calculate  differences from start and mean prices
    price_features["norm_diff_exp_mean_price"] = (
        price_features["item_price"] - price_features["exp_mean_price"]
    ) / price_features["exp_mean_price"]
    price_features["norm_diff_start_price"] = (
        price_features["item_price"] - price_features["start_price"]
    ) / price_features["start_price"]
    # Calculate normalized differences from start and mean prices
    price_features["norm_diff_exp_mean_price"] = (
        price_features["item_price"] - price_features["exp_mean_price"]
    ) / price_features["exp_mean_price"]
    # Calculate normalized differenced from mean category price per month
    price_features = price_features.merge(
        items[["item_id", "item_category_id"]], how="left", on="item_id"
    )
    price_features["norm_diff_cat_price"] = price_features.groupby(
        ["date_block_num", "item_category_id"]
    )["item_price"].transform(lambda x: (x - x.mean()) / x.mean())
    # Retain only the necessary features
    price_features = price_features[
        [
            "date_block_num",
            "item_id",
            "item_price",
            "norm_diff_exp_mean_price",
            "norm_diff_start_price",
            "norm_diff_cat_price",
        ]
    ]

    features = [
        "item_price",
        "norm_diff_exp_mean_price",
        "norm_diff_start_price",
        "norm_diff_cat_price",
    ]
    newnames = ["last_" + f for f in features]
    aggs = {f: "last" for f in features}
    renames = {f: "last_" + f for f in features}

    features = []
    for dbn in tqdm(range(1, 35)):
        f_temp = (
            price_features.query(f"date_block_num<{dbn}")
            .groupby("item_id")
            .agg(aggs)
            .rename(columns=renames)
        )
        f_temp["date_block_num"] = dbn
        features.append(f_temp)
    features = pd.concat(features).reset_index()

    matrix = matrix.merge(features, on=["date_block_num", "item_id"], how="left")
    matrix[newnames] = matrix[newnames].fillna(999999)
    return matrix


matrix = add_price_features(matrix, train)

## Categorical features

Add item categories and supercategories, e.g. "video games", "music", "PS4" (uses custom item categories dataframe with manually-chosen categories)

In [ ]:
matrix = matrix.merge(items.drop(columns='item_name'), on='item_id', how='left')
matrix = matrix.merge(item_categories_extra, on='item_category_id', how='left')

City that the shop is located in, extracted from the shop name
(from https://www.kaggle.com/dlarionov/feature-engineering-xgboost)

In [ ]:
def add_city_codes(matrix, shops):
    shops.loc[
        shops.shop_name == 'Сергиев Посад ТЦ "7Я"', "shop_name"
    ] = 'СергиевПосад ТЦ "7Я"'
    shops["city"] = shops["shop_name"].str.split(" ").map(lambda x: x[0])
    shops.loc[shops.city == "!Якутск", "city"] = "Якутск"
    shops["city_code"] = shops["city"].factorize()[0]
    shop_labels = shops[["shop_id", "city_code"]]
    matrix = matrix.merge(shop_labels, on='shop_id', how='left')
    return matrix

matrix = add_city_codes(matrix, shops)

In [ ]:
gc.collect()
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
# matrix.to_pickle("matrixcheckpoint_1.pk1")
print("Saved matrixcheckpoint 1")

Simple lags features, i.e. the value of a feature for the same item-shop combination from n months back

### Rolling mean features
Sales from previous months are a good predictor of future sales, but chance fluctuations mean that an average of several previous months may be more reliable than counts from a single month. Pandas has several windowing functions for time series built in, 3 of which are demonstrated below : expanding (all previous timepoints), rolling (a fixed number of previous timepoints) and exponential (a weighted window with weights which decrease with time distance before the current point). 

In [ ]:
shop_id = 16
item_id = 482
im = matrix.query(f"shop_id=={shop_id} & item_id=={item_id}")[['date_block_num', 'item_cnt_month']]
im['moving average'] = im['item_cnt_month'].ewm(halflife=1).mean()
im['expanding'] = im['item_cnt_month'].expanding().mean()
im['rolling_12'] = im['item_cnt_month'].rolling(window=12, min_periods=1).mean()
im = im.set_index('date_block_num')
ax = im.plot(figsize=(12,5), marker='.', title='Time series averaging methods')

A weakness of the pandas windowed functions is that they do not insert zeros for missing timepoints, which is a problem when calculating values for low-volume items which do not record a sale in some months of their availability. This is solved here by creating a sales matrix which contains entries for all items in all months, and using this to generate windowed features. Entries for months prior to an items availability are set to NaN so that they are not included in calculations.

In [ ]:
def create_full_train_test(sales_train, test, clip_target=20.0):
    # Create a train set with all items and all shops in all date blocks.
    # Add revenue column to sales_train
    sales_train["item_revenue_day"] = sales_train["item_price"] * sales_train["item_cnt_day"]
    # Aggregate item_id / shop_id item_cnts and revenue at the month level
    sales_train_grouped = sales_train.groupby(["date_block_num", "shop_id", "item_id"]).agg(
        item_cnt_month=pd.NamedAgg(column="item_cnt_day", aggfunc="sum"),
        item_revenue_month=pd.NamedAgg(column="item_revenue_day", aggfunc="sum"),
    )
    # Create sets of items, shops and date blocks
    item_ids = set(sales_train.item_id).union(set(test.item_id))
    shop_ids = set(sales_train.shop_id).union(set(test.shop_id))
    date_block_nums = set(sales_train.date_block_num).union(set([34]))
    # Create all permutations as indexes
    indexdataframe = pd.DataFrame(
        np.array(list(itertools.product(date_block_nums, item_ids, shop_ids))),
        columns=["date_block_num", "item_id", "shop_id"],
    )
    m = indexdataframe.merge(sales_train_grouped, how="left", on=["date_block_num", "item_id", "shop_id"])
    m.item_cnt_month = m.item_cnt_month.fillna(0)
    m.item_revenue_month = m.item_revenue_month.fillna(0)
    shop_first_sales = train.groupby("shop_id").date_block_num.min().rename("shop_first_month")
    shop_last_sales = train.groupby("shop_id").date_block_num.max().rename("shop_last_month")
    item_first_sales = train.groupby("item_id").date_block_num.min().rename("item_first_month")
    m = m.merge(shop_first_sales, on="shop_id", how="left")
    m = m.merge(shop_last_sales, on="shop_id", how="left")
    m = m.merge(item_first_sales, on="item_id", how="left")
    mask = (
        (m.date_block_num < m.shop_first_month)
        | (m.date_block_num < m.item_first_month)
        | (m.date_block_num > m.shop_last_month)
    )
    m = m.drop(columns=["shop_first_month", "shop_last_month", "item_first_month"])
    m.loc[mask, ["item_cnt_month", "item_revenue_month"]] = np.nan
    
    if clip_target is not None:
        m['item_cnt_month'] = m['item_cnt_month'].clip(upper=clip_target)
    m = fu.reduce_mem_usage(m)
    return m

In [ ]:
m = create_full_train_test(train, test)

Define general purpose rolling mean function

In [ ]:
import gc

def add_rolling_ME(
    matrix,
    features,
    window=3,
    min_periods=1,
    ewm=False,
    source_matrix=None,
    target_feature="item_cnt_month",
    aggregation="mean",
    rolling_aggregation="mean",
    expanding=False,
):
    """Add a rolling mean item_cnt_month feature for a specificed categorical feature
    or features. Calculates using the feature matrix and therefore does not insert zeros
    for features with zero sales in a specific month (use feature specific rolling ME
    functions for rolling item or item-shop features)"""
    features = fu.list_if_not(features)
    if source_matrix is None:
        source_matrix = matrix
    im = source_matrix.groupby(features + ["date_block_num"])[target_feature].agg(aggregation)
    im = im.reset_index(level=features + ["date_block_num"])
    im = fu.reduce_mem_usage(im)
    if ewm:
        feat_name = f"{'_'.join(features)}_{target_feature}_{aggregation}_ewm_hl_{window}"
        print(f'Creating feature "{feat_name}"')
        im[feat_name] = im.groupby(features)[target_feature].transform(lambda x: x.ewm(halflife=window, min_periods=min_periods).mean())
    elif expanding:
        feat_name = f"{'_'.join(features)}_{target_feature}_{aggregation}_expanding_{rolling_aggregation}"
        print(f'Creating feature "{feat_name}"')
        im[feat_name] = im.groupby(features)[target_feature].transform(lambda x: x.expanding(min_periods=min_periods).aggregate(rolling_aggregation))
    else:
        feat_name = (
            f"{'_'.join(features)}_{target_feature}_{aggregation}_rolling_{rolling_aggregation}_win_{window}"
        )
        print(f'Creating feature "{feat_name}"')
        im[feat_name] = im.groupby(features)[target_feature].transform(lambda x: x.rolling(window=window, min_periods=min_periods).aggregate(rolling_aggregation))
    im = im.drop(columns=target_feature)
    im.loc[:, 'date_block_num'] += 1
    matrix = matrix.merge(im, on=['date_block_num'] + features, how='left')
    return matrix

Implement rolling mean / median / minimum / maximum features

In [ ]:
# Shop-item specific windowed features
matrix = add_rolling_ME(matrix, features=["shop_id", "item_id"], window=12, source_matrix=m)
gc.collect()
matrix = add_rolling_ME(matrix, features=["shop_id", "item_id"], window=1, ewm=True, source_matrix=m)
gc.collect()
matrix = add_rolling_ME(matrix, features=["shop_id", "item_id"], window=1, ewm=True, source_matrix=m, aggregation="median")
gc.collect()
# Item all shops features, various stats
matrix = add_rolling_ME(matrix, features=["item_id"], window=12, source_matrix=m)
gc.collect()
matrix = add_rolling_ME(matrix, features=["item_id"], window=1, ewm=True, source_matrix=m)
gc.collect()
matrix = add_rolling_ME(
    matrix, ["item_id"], window=12, target_feature="item_revenue_month", aggregation="sum", 
)
gc.collect()
matrix = add_rolling_ME(matrix, features=["item_id"], window=12, source_matrix=m, aggregation="median")
gc.collect()
matrix = add_rolling_ME(
    matrix, features=["item_id"], window=1, ewm=True, source_matrix=m, aggregation="median"
)
gc.collect()
matrix = add_rolling_ME(matrix, features=["item_id"], window=12, source_matrix=m, aggregation="max")
gc.collect()
matrix = add_rolling_ME(matrix, features=["item_id"], window=12, source_matrix=m, aggregation="var")
gc.collect()
del(m)
print("Rolling item features created")

In [ ]:
# The item age max 12 feature is used for mean encoding features later
matrix["item_age_max_12"] = matrix["item_age"].clip(upper=12)

# Item category rolling feature (necessary for ratio feature later)
matrix = add_rolling_ME(matrix, ["item_category_id"], window=12)

# Shop - item category interaction features
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=12)
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=1, ewm=True)
matrix = add_rolling_ME(
    matrix,
    ["shop_id", "item_category_id"],
    window=12,
    target_feature="item_revenue_month",
    aggregation="sum",
)
gc.collect()
# Shop features
matrix = add_rolling_ME(matrix, ["shop_id"], window=12)
matrix = add_rolling_ME(
    matrix, ["shop_id"], window=12, target_feature="item_revenue_month", aggregation="sum"
)
# Shop digital features (almost equivalent to the shop features)
matrix = add_rolling_ME(matrix, ["shop_id", "digital"], window=12)
gc.collect()
# Item age - item category 
matrix = add_rolling_ME(matrix, ["item_age_max_12", "item_category_id"], window=1, ewm=True)
matrix = add_rolling_ME(matrix, ["item_age_max_12", "item_category_id"], window=12)
matrix = add_rolling_ME(
    matrix,
    ["item_age_max_12", "item_category_id"],
    window=12,
    target_feature="item_revenue_month",
    aggregation="sum",
)
gc.collect()
# Item age - shop - category interaction rolling mean
matrix = add_rolling_ME(matrix, ["item_age_max_12", "shop_id", "item_category_id"], window=1, ewm=True)
matrix = add_rolling_ME(matrix, ["item_age_max_12", "shop_id", "item_category_id"], window=12)
matrix = add_rolling_ME(
    matrix,
    ["item_age_max_12", "shop_id", "item_category_id"],
    window=12,
    target_feature="item_revenue_month",
    aggregation="sum",
)

# Item price rolling mean
matrix = add_rolling_ME(matrix, ["item_id"], window=1, target_feature="last_item_price", ewm=True)

# Shop - item category interaction features
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=1, ewm=True, aggregation="median")

# New item - item category mean encode
matrix = add_rolling_ME(matrix, ["item_age_max_12", "item_category_id"], window=12, aggregation="median")

# Shop - item category interaction features
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=0.5, ewm=True, aggregation="min")

# Shop - item category interaction features
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=12, aggregation="max")
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id"], window=0.5, ewm=True, aggregation="max")

# New item - item category mean encode
matrix = add_rolling_ME(matrix, ["item_age_max_12", "item_category_id"], window=1, ewm=True, aggregation="max")
matrix = add_rolling_ME(matrix, ["item_age_max_12", "item_category_id"], window=6, aggregation="max")
gc.collect()

Rolling variance features

In [ ]:
# New item - item category mean
matrix = add_rolling_ME(
    matrix,
    ["item_age_max_12", "item_category_id"],
    window=12,
    rolling_aggregation="var"
)

# New item - shop - category interaction rolling mean
matrix = add_rolling_ME(matrix, ["item_age_max_12", "shop_id", "item_category_id"], window=12, rolling_aggregation="var")
matrix = add_rolling_ME(
    matrix,
    ["item_age_max_12", "shop_id", "item_category_id"],
    window=12,
    target_feature="item_revenue_month",
    aggregation="sum",
    rolling_aggregation="var"
)

Expanding sum features (also functions as a marker of a previous item sale)

In [ ]:
matrix = add_rolling_ME(
    matrix,
    ["shop_id", "item_id"],
    target_feature="item_cnt_month",
    aggregation="sum",
    rolling_aggregation="sum",
    expanding=True,
)
matrix = add_rolling_ME(
    matrix,
    ["item_id"],
    target_feature="item_revenue_month",
    aggregation="sum",
    rolling_aggregation="sum",
    expanding=True,
)

More expanding mean features

In [ ]:
matrix = add_rolling_ME(
    matrix,
    ["month", "item_category_id"],
    expanding=True,
)
matrix = add_rolling_ME(
    matrix,
    ["month", "supercategory_id"],
    expanding=True,
)
matrix = add_rolling_ME(
    matrix,
    ["month", "shop_id"],
    expanding=True,
)
gc.collect()

Mean sales of same category features with the same age in past data, using expanding mean

In [ ]:
matrix = add_rolling_ME(matrix, ["shop_id", "supercategory_id", "item_age_max_12"], expanding=True)
matrix = add_rolling_ME(matrix, ["supercategory_id", "item_age_max_12"], expanding=True)
matrix = add_rolling_ME(matrix, ["shop_id", "item_category_id", "item_age_max_12"], expanding=True)
matrix = add_rolling_ME(matrix, ["item_category_id", "item_age_max_12"], expanding=True)

In [ ]:
gc.collect()
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
# matrix.to_pickle("matrixcheckpoint_2.pk1")
print("Saved matrixcheckpoint 2")

In [ ]:
def simple_lag_feature(matrix, lag_feature, lags):
    targetseries = matrix.loc[:,['date_block_num', 'item_id', 'shop_id'] + [lag_feature]]
    targetseries = targetseries.set_index(['date_block_num', 'item_id', 'shop_id'])
    targetseries = targetseries[lag_feature]
    for lag in tqdm(lags):
        matrix = fu.add_lag_feature(matrix, targetseries, ['item_id', 'shop_id'], lag, fillna=0)
    return matrix

In [ ]:
matrix = simple_lag_feature(matrix, 'item_cnt_month', lags=[1,2,3])
matrix = simple_lag_feature(matrix, 'item_cnt_month_unclipped', lags=[1])
matrix = simple_lag_feature(matrix, 'item_cnt_day_avg', lags=[1, 2])
matrix = simple_lag_feature(matrix, 'item_revenue_month', lags=[1])
gc.collect()
print("Lag features created")

Mean encoding, i.e. the mean value of a target feature for each level of a categorical feature or combination of categorical features

In [ ]:
def create_apply_ME(
    matrix, grouping_fields, name, lags=[1], source_frame=None, target="item_cnt_month", aggregation="mean",
):
    if source_frame is None:
        source_frame = matrix
    grouping_fields = fu.list_if_not(grouping_fields)
    me_series = (
        source_frame.groupby(["date_block_num"] + grouping_fields)[target].agg(aggregation).rename(name)
    )
    matrix = fu.apply_lags(matrix, me_series, grouping_fields, lags)
    return matrix

In [ ]:
matrix = create_apply_ME(matrix, ["item_id"], "item_id_ME", lags=[1])
matrix = create_apply_ME(matrix, ["item_id"], "item_id_item_cnt_day_avg", lags=[1,2,3], target="item_cnt_day_avg")

In [ ]:
matrix = create_apply_ME(matrix, ["item_id"], "item_id_ME_unclipped", target="item_cnt_month_unclipped")
matrix = create_apply_ME(matrix, ["item_category_id"], "item_category_id_ME", [1, 2, 12])
matrix = create_apply_ME(matrix, ["supercategory_id"], "supercategory_id_ME", [1, 12])
matrix = create_apply_ME(matrix, ["platform_id"], "platform_id_ME", [1, 12])
matrix = create_apply_ME(matrix, 
    ["shop_id", "item_category_id"], "shop_id_item_category_id_ME", [1]
)
matrix = create_apply_ME(matrix, ["shop_id", "digital"], "shop_id_digital_ME", [1])
matrix = create_apply_ME(matrix, ["city_code", "item_id"], "city_code_item_id_ME", [1], target="item_cnt_day_avg")
matrix["item_age_max_12"] = matrix["item_age"].clip(upper=12)
matrix = create_apply_ME(matrix, ["item_age_max_12", "item_category_id"], "item_age_item_category_id_ME", [1], target='item_cnt_month')
matrix = create_apply_ME(matrix, ["item_age_max_12", "supercategory_id"], "new_item_supercategory_id_ME", [1])
matrix = create_apply_ME(matrix, ["item_age_max_12", "shop_id", "item_category_id"], "new_item_shop_id_item_category_id_ME", [1])

### Ratios between recent sales and rolling 12 month means

In [ ]:
matrix["item_id_item_cnt_1_12_ratio"] = (
    matrix["item_id_ME_lag_1"] / matrix["item_id_item_cnt_month_mean_rolling_mean_win_12"]
).fillna(1)
matrix["item_category_id_item_cnt_lag_1_12_ratio"] = (
    matrix["item_category_id_ME_lag_1"] / matrix["item_category_id_item_cnt_month_mean_rolling_mean_win_12"]
).fillna(1)
matrix["shop_id_item_category_id_item_cnt_lag_1_12_ratio"] = (
    matrix["shop_id_item_category_id_ME_lag_1"]
    / matrix["shop_id_item_category_id_item_cnt_month_mean_rolling_mean_win_12"]
).fillna(1)

In [ ]:
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
gc.collect()
print("Mean encoding features created")

Percentage change in an aggregate feature over a specified period, e.g. percentage change in total shop revenue compared to the previous month

In [ ]:
def add_pct_change(
    df, group_feats, quantity="item_cnt_month", agg_function="mean", periods=1, lag=1, clip_value=None,
):
    """
    Adds a column of month-to-month proportion change values for an aggregate of a feature
    
    Arguments:
    df (Dataframe): dataframe to add feature to
    group_feats (str or list): column or list of columns to group items by when generating
    aggregate features
    quantity (str): column to aggregate
    agg_function (str): aggregate function to use, passed to pandas GroupBy.agg() method
    periods (int or str): time deltas to use when calculating percentage changes of the
    aggregate feature
    lag (int): lag value for joining the new feature to the dataframe
    clip_value: positive absolute value used to set the negative and positive maximum
    values of the new feature (delta values can be very large due to zero values)
    """
    # Put string arguments for arguments "periods" and "group_feats" into lists
    periods = fu.list_if_not(periods, int)
    group_feats = fu.list_if_not(group_feats)
    # group_feats_full: list of grouping columns + time column
    group_feats_full = ["date_block_num"] + group_feats
    # Create a template index of all group feature values in all months (original df misses levels in months where they have no sales)
    idx = pd.MultiIndex.from_product([df[col].unique() for col in group_feats_full], names=group_feats_full)
    template = pd.DataFrame(index=idx)
    template = template.sort_index()
    # Create aggregate feature and merge the results with the template
    aggs = df.groupby(group_feats_full)[quantity].agg(agg_function).fillna(value=0)
    template = template.merge(aggs, on=group_feats_full, how="left")
    # Generate pct_change feature for each specified period, clip values then add to df with a lag of 1
    for period in periods:
        feat_name = "_".join(group_feats + [quantity] + [agg_function] + ["delta"] + [str(period)])
        print(f"Adding feature {feat_name}")
        feature_series = (
            template.groupby(group_feats)[quantity]
            .transform(lambda x: x.pct_change(periods=period, fill_method="pad"))
            .fillna(value=0)
            .rename(feat_name)
        )
        if clip_value is not None:
            feature_series = feature_series.clip(lower=-clip_value, upper=clip_value)
        # Add to df with a specified lag
        df = fu.add_lag_feature(df, feature_series, group_feats, lag=lag)
    gc.collect()
    return df

In [ ]:
# Mean item price change
matrix = add_pct_change(matrix, ["item_id"], "last_item_price", clip_value=3, lag=0)
# Mean item sales change
matrix = add_pct_change(matrix, ["item_id"], "item_cnt_month", clip_value=3)
# Mean item category sales change
matrix = add_pct_change(matrix, ["item_category_id"], "item_cnt_month", clip_value=3)
# Mean sales change across everything
matrix = add_pct_change(matrix, ["shop_id", "item_id"], "item_cnt_month", clip_value=3)
# Mean item revenue change
matrix = add_pct_change(matrix, ["item_id"], "item_revenue_month", agg_function="sum", clip_value=3)
# Delta 1 features lagged by 12 months, intended to capture seasonal trends
matrix = add_pct_change(matrix, ["item_category_id"], "item_cnt_month", lag=12, clip_value=3,)
matrix = add_pct_change(matrix, ["shop_id"], "item_cnt_month", lag=12, clip_value=3)

In [ ]:
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
gc.collect()
print("Delta features created")

### Categorical interaction features

In [ ]:
def interaction_feature(features):
    feature_name = '_'.join(['interaction'] + features)
    matrix[feature_name] = matrix[features[0]].apply(str)
    for feature in features[1:]:
        matrix[feature_name] = matrix[feature_name] + '_' +  matrix[feature].apply(str)
    matrix[feature_name] = fu.reduce_mem_usage(matrix[feature_name], allow_categorical=False)
    return matrix

In [ ]:
# Month category interactions
interaction_feature(["month", "item_category_id"])
interaction_feature(["month", "supercategory_id"])
interaction_feature(["month", "digital"])
# New item category interactions
interaction_feature(["new_item", "item_category_id"])
interaction_feature(["new_item", "supercategory_id"])
interaction_feature(["new_item", "platform_id"])
# Shop - category interactions
interaction_feature(["shop_id", "item_category_id"])
# Digital shop_id interactions
matrix["is_shop_55"] = matrix.shop_id == 55
interaction_feature(["is_shop_55", "digital"])
matrix = matrix.drop(columns=["is_shop_55"])
matrix = matrix.rename(columns={"interaction_is_shop_55_digital":"interaction_shop_digital"})

### Artist name feature for music categories  

The "item_name" field of items in the music categories typically begin with the artist's name marked with one of three patterns: either all uppercase, separated from the release title by a doublespace, or separated by dot-space (. )

In [ ]:
items.query("item_category_id==55").head(5)

Using regex operations, these are extracted and made more homogenous by removal of special characters and converting to uppercase. This method is not perfect and sometimes extracts general terms such as "Jazz" instead, but hopefully is useful. Artist names are also count encoded according to their occurences in the items table.

In [ ]:
def add_name_features(
    matrix,
    factorize=True,
    encode_missing=float("nan"),
    unique_thresh=2,
    items=items,
    fillna_value=None,
    feature_name="music_artist",
):
    # This extracts artist names for music categories and adds them as a feature.
    def extract_artist(st):
        import re

        st = st.strip()
        if st.startswith("V/A"):
            artist = "V/A"
        elif st.startswith("СБ"):
            artist = "СБ"
        else:
            # Retrieves artist names using the double space or all uppercase pattern
            mus_artist_dubspace = re.compile(r".{2,}?(?=\s{2,})")
            match_dubspace = mus_artist_dubspace.match(st)
            mus_artist_capsonly = re.compile(r"^([^a-zа-я]+\s)+")
            match_capsonly = mus_artist_capsonly.match(st)
            candidates = [match_dubspace, match_capsonly]
            candidates = [m[0] for m in candidates if m is not None]
            # Sometimes one of the patterns catches some extra words so choose the shortest one
            if len(candidates):
                artist = min(candidates, key=len)
            else:
                # If neither of the previous patterns found something, use the dot-space pattern
                mus_artist_dotspace = re.compile(r".{2,}?(?=\.\s)")
                match = mus_artist_dotspace.match(st)
                if match:
                    artist = match[0]
                else:
                    artist = ""
        artist = artist.upper()
        artist = re.sub(r"[^A-ZА-Я ]||\bTHE\b", "", artist)
        artist = re.sub(r"\s{2,}", " ", artist)
        artist = artist.strip()
        return artist

    music_categories = [55, 56, 57, 58, 59, 60]
    items.loc[items.item_category_id.isin(music_categories), feature_name] = items.loc[
        items.item_category_id.isin(music_categories), "item_name"
    ].apply(extract_artist)

    items.loc[items[feature_name] == "", feature_name] = float("nan")

    freq_enc_name = feature_name + "_freq_encode"
    # Remove artist names which are associated with less than unique_thresh items
    n_items = items[feature_name].value_counts().rename(freq_enc_name)
    items = items.merge(n_items, left_on=feature_name, right_index=True, how="left")
    items = items.loc[items[freq_enc_name] >= unique_thresh, :]

    if factorize:
        items[feature_name] = items[feature_name].factorize(na_sentinel=-1)[0]
        items.loc[items[feature_name] == -1, feature_name] = encode_missing

    matrix = matrix.merge(
        items.loc[items.item_category_id.isin(music_categories), ["item_id", feature_name, freq_enc_name],],
        on="item_id",
        how="left",
    )
    if fillna_value is not None:
        matrix[[feature_name, freq_enc_name]] = matrix[[feature_name, freq_enc_name]].fillna(fillna_value)
    return matrix

In [ ]:
matrix = add_name_features(matrix, factorize=True, fillna_value=-1)
# Music artist rolling mean sale count and revenue *** ADD EXPANDING ***
matrix = add_rolling_ME(matrix, ["music_artist"], window=12, target_feature="item_cnt_month")
matrix = add_rolling_ME(
    matrix, ["music_artist"], window=24, target_feature="item_revenue_month", aggregation="sum"
)
print("Created categorical features")

In [ ]:
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)

## Minor data leak features  
i.e. counts of new and unique items in the current month  
This counts the number of unique and new items sold in the current month and also category, which could have a positive relationship to sales. As this can also be calculated for the test set (assuming that the set of test items is the set of items that were sold in the test month) this is a kind of data leak. 

In [ ]:
def add_unique_item_features(matrix):
    # Adds the number of unique and new items per month and month-category, and these values as a proportion
    unique_id_features = (
        matrix.groupby(["date_block_num", "item_category_id"])
        .item_id.nunique()
        .rename("unique_items_cat")
        .reset_index()
    )
    unique_id = (
        unique_id_features.groupby("date_block_num").unique_items_cat.sum().rename("unique_items_month")
    )
    unique_id_features = unique_id_features.merge(
        unique_id, how="left", left_on="date_block_num", right_index=True,
    )
    unique_id_features["cat_items_proportion"] = unique_id_features["unique_items_cat"].div(
        unique_id_features["unique_items_month"]
    )
    new_items_cat = (
        matrix.loc[matrix.new_item == 1, ["date_block_num", "item_id", "item_category_id"]]
        .groupby(["date_block_num", "item_category_id"])
        .item_id.nunique()
        .rename("new_items_cat")
    )
    unique_id_features = unique_id_features.merge(
        new_items_cat, left_on=["date_block_num", "item_category_id"], right_index=True, how="left",
    )
    unique_id_features["new_items_cat"] = unique_id_features["new_items_cat"].fillna(0)
    new_items_month = (
        unique_id_features.groupby("date_block_num").new_items_cat.sum().rename("new_items_month")
    )
    unique_id_features = unique_id_features.merge(
        new_items_month, left_on="date_block_num", right_index=True, how="left"
    )
    unique_id_features["cat_items_new_proportion"] = (
        unique_id_features["new_items_cat"] / unique_id_features["unique_items_cat"]
    )
    matrix = matrix.merge(unique_id_features, on=["date_block_num", "item_category_id"], how="left")
    return matrix


matrix = add_unique_item_features(matrix)

In [ ]:
gc.collect()
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
# matrix.to_pickle("matrixcheckpoint_3.pk1")
print("Saved matrixcheckpoint 3")

Sales of item with item_id one above or below (these tend to be related items)

In [ ]:
def add_neighbor_item_features(matrix, feature, group= ['item_id']):
    # Item_id minus 1
    f = matrix.groupby(["date_block_num"] + group)[feature].agg("mean")
    name_minus = "minus1_" + feature
    f.name = name_minus
    f = f.reset_index(["date_block_num"] + group)
    f["item_id"] = f["item_id"] + 1
    f['date_block_num'] += 1
    matrix = matrix.merge(f, on=["date_block_num"] + group, how='left')
    matrix[name_minus] = matrix[name_minus].fillna(99)
#     f = f.set_index(["date_block_num"] + group)[new_name]
#     f = f.reset_index(["date_block_num"] + group)
#     matrix = fu.add_lag_feature(matrix, f, group, lag=0)
#     matrix = matrix.rename(columns={new_name + '_lag_0': new_name})
    # Item_id plus 1
    name_plus = "plus1_" + feature
    f = f.rename(columns={name_minus:name_plus})
    f["item_id"] = f["item_id"] - 2
    f['date_block_num']
    matrix = matrix.merge(f, on=["date_block_num"] + group, how='left')
    matrix[name_plus] = matrix[name_plus].fillna(99)
#     f = f.set_index(["date_block_num"] + group)[new_name]
#     matrix = fu.add_lag_feature(matrix, f, group, lag=0)
#     matrix = matrix.rename(columns={new_name + '_lag_0': new_name})
    return matrix

In [ ]:
features = ["item_cnt_day_avg_lag_1", 'item_id_item_cnt_day_avg_lag_1', 'item_id_item_cnt_month_mean_rolling_mean_win_12']
for feature in features:
    matrix = add_neighbor_item_features(matrix, feature)

Item name length as a feature.

In [ ]:
import re
def clean_item_name(string):
    # Removes bracketed terms, special characters and extra whitespace
    string = re.sub(r"\[.*?\]", "", string)
    string = re.sub(r"\(.*?\)", "", string)
    string = re.sub(r"[^A-ZА-Яa-zа-я0-9 ]", "", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = string.lower()
    return string

items["item_name_cleaned_length"] = items["item_name"].apply(clean_item_name).apply(len)
items['item_name_length'] = items.item_name.apply(len)
matrix = matrix.merge(items[['item_id', 'item_name_length', 'item_name_cleaned_length']], how='left', on='item_id')

In [ ]:
gc.collect()
matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
matrix.to_pickle("matrixcheckpoint_4.pk1")
print("Saved matrixcheckpoint 4")

## Similar item name values

(mention new items) this uses the fuzzy string matching package FuzzyWuzzy to find items with similar names and return sales count values for these items in their first month of availability, weighted by their similarity to the target name. Bracketed terms are stripped from the item names before matching, because these tend to contain product codes or format information (e.g. DVD, Xbox). 

In [ ]:
# items.loc[[23, 34, 12],:]

In [ ]:
# def item_name_similarity_frame(itemsdf):
#     # Makes a dataframe of similarity values between item names calculated with FuzzyWuzzy
#     # itemsdf must have a supercategory_id field
#     import re
#     from itertools import combinations

#     import scipy.sparse as sp
#     from fuzzywuzzy import fuzz

#     def strip_sq_brackets(string):
#         return re.sub(r"\[.*?\]", "", string)

#     def strip_rd_brackets(string):
#         return re.sub(r"\(.*?\)", "", string)

#     items["item_name"] = items["item_name"].apply(strip_sq_brackets).apply(strip_rd_brackets)

#     itnames = items.item_name.to_list()
#     itsupcats = items.supercategory_id.to_list()
#     pairs = combinations(items.index, 2)
#     sims = sp.dok_matrix((len(itnames), len(itnames)), dtype=np.int8)

#     for id1, id2 in tqdm(
#         pairs, total=len(itnames) * (len(itnames) - 1) / 2, desc="Calculating similarity values"
#     ):
#         if itsupcats[id1] != itsupcats[id2]:
#             pass
#         else:
#             sims[id1, id2] = fuzz.token_sort_ratio(itnames[id1], itnames[id2], force_ascii=False)

#     csims = sims.tocsr()
#     csims = csims + csims.T
#     csims = pd.DataFrame.sparse.from_spmatrix(csims)
#     return csims


# def make_sim_item_features(matrix, sim_frame, return_fields, sim_thresh=50, max_n=5, max_item_age=12):
#     storelist = []
#     mean_fields = [s + "_all_shops" for s in return_fields]
#     for date_block_num in tqdm(range(3, 35), "Generating similar item name features"):
#         for item_age in np.sort(matrix.loc[matrix.date_block_num == date_block_num, "item_age"].unique()):

#             def get_sim_item_features(item_ids):
#                 slist = []
#                 for item_id in item_ids:
#                     sim_items = sim_frame.loc[item_id, item_ids_past].nlargest(max_n)
#                     sim_items = sim_items[sim_items>sim_thresh]
#                     if len(sim_items)==0:
#                         pass
#                     else:
#                         sim_item_values = (
#                             past_months.loc[(slice(None), sim_items.index), return_fields]
#                             .groupby("shop_id")[return_fields]
#                             .mean()
#                         )
#                         sim_item_values[mean_fields] = sim_item_values.mean()
#                         sim_item_values["item_id"] = item_id
#                         slist.append(sim_item_values)
#                 sframe = pd.concat(slist)
#                 sframe["date_block_num"] = date_block_num
#                 sframe = sframe.reset_index()
#                 return sframe
#             if item_age < date_block_num:
#                 if item_age >= max_item_age:
#                     past_months = (
#                         matrix.query(f"date_block_num<{date_block_num} & date_block_num>1 & item_age>={item_age}")
#                         .groupby(["shop_id", "item_id"])[return_fields]
#                         .mean()
#                     )
#                     item_ids_past = past_months.index.get_level_values("item_id").unique()
#                     item_ids = matrix.query(
#                         f"date_block_num=={date_block_num} & item_age>={item_age}"
#                     ).item_id.unique()
#                     sframe_new = get_sim_item_features(item_ids)
#                     storelist = storelist + [sframe_new]
#                     break
#                 else:
#                     past_months = (
#                         matrix.query(f"date_block_num<{date_block_num} & date_block_num>1 & item_age=={item_age}")
#                         .groupby(["shop_id", "item_id"])[return_fields]
#                         .mean()
#                     )
#                     item_ids_past = past_months.index.get_level_values("item_id").unique()
#                     item_ids = matrix.query(
#                         f"date_block_num=={date_block_num} & item_age=={item_age}"
#                     ).item_id.unique()
#                     sframe_new = get_sim_item_features(item_ids)
#                     storelist = storelist + [sframe_new]
#     sim_item_features = pd.concat(storelist)
#     sim_item_field_names = {s: "sim_item_name_" + s for s in return_fields + mean_fields}
#     sim_item_features = sim_item_features.rename(columns=sim_item_field_names)
#     return sim_item_features


# def add_sim_item_features(matrix, sim_item_features, fill_na_val=None):
#     oldcols = matrix.columns
#     matrix = matrix.merge(sim_item_features, on=["date_block_num", "shop_id", "item_id"], how="left")
#     newcols = matrix.columns.difference(oldcols)
#     if fill_na_val is not None:
#         matrix[newcols] = matrix[newcols].fillna(fill_na_val)
#     return matrix

Add the similar item name sales features. Creating these takes ~10 hours on an i7 laptop, so previously created features can be loaded instead.

In [ ]:
# load_data = "all"  # "sim_matrix" to load the name similarity matrix then compute new features, and "all" to load the features.
# if load_data not in ["all", "sim_matrix"]:
#     items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
#     items = items.merge(
#         item_categories_extra[["item_category_id", "supercategory_id"]], on="item_category_id", how="left"
#     )
#     sim_matrix = item_name_similarity_frame(items)
# elif load_data == "sim_matrix":
#     sim_matrix = pd.read_pickle("../input/predict-future-sales-extra/sim_frame_no_brackets.pk1")
#     sim_item_features = make_sim_item_features(matrix, sim_matrix, ["item_cnt_month", "item_cnt_day_avg"], sim_thresh=70, max_n=5)
#     sim_item_features.to_csv("sim_item_features_w_missing.csv.gz")
# elif load_data == "all":
#     sim_item_features = pd.read_csv("../input/predict-future-sales-extra/sim_item_features_w_missing.csv")

# matrix = add_sim_item_features(matrix, sim_item_features)

# missingmask = matrix["sim_item_name_item_cnt_month"].isna()
# matrix.loc[missingmask, "sim_item_name_item_cnt_month"] = matrix.loc[
#     missingmask, "shop_id_supercategory_id_item_age_max_12_item_cnt_month_mean_expanding_mean"
# ]
# missingmask = matrix["sim_item_name_item_cnt_month_all_shops"].isna()
# matrix.loc[missingmask, "sim_item_name_item_cnt_month_all_shops"] = matrix.loc[
#     missingmask, "supercategory_id_item_age_max_12_item_cnt_month_mean_expanding_mean"
# ]
# del(sim_item_features)
# gc.collect()

Use the mean for the supercategory / age combination to fill in missing similar name sales values for cases when no similar names were found, additionally leave the supercategory / age mean as a new feature.

In [ ]:
# matrix, oldcols = fu.shrink_mem_new_cols(matrix, oldcols)
# matrix.to_pickle("matrixcheckpoint_5.pk1")

In [ ]:
%reset -f

# Model fitting section

In [ ]:
import gc
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

import lightgbm as lgbm

In [ ]:
import pandas as pd

In [ ]:
matrix = pd.read_pickle("matrixcheckpoint_4.pk1")

### Generate train, validation, test sets from feature matrix

In [ ]:
def train_test_x_y(
    matrix, test_month, keep_from_month=3, new_target=None
):
    if new_target is not None:
        matrix["item_cnt_month"] = matrix.loc[:, new_target]
        matrix = matrix.drop(columns=new_target)

    def split_train_test(matrix, test_month=33):
        # Split the matrix into train and test sets.
        test = matrix.loc[matrix.date_block_num==test_month, :]
        train = matrix.loc[matrix.date_block_num < test_month, :]
        return train, test

    def xysplit(matrix):
        # Split a train and test set into into x and y sets, with item_cnt as the target y variable
        y = matrix.item_cnt_month
        X = matrix.drop(columns=["item_cnt_month"])
        return (X, y)

    matrix = matrix.drop(
        columns=[
            "item_revenue_month",
            "item_price",
            "item_cnt_month_original",
            "item_cnt_month_unclipped",
            "item_cnt_day_avg",
            "new_item",
            "new_shop",
            "item_age",
            "shop_age",
            "digital",
        ],
        errors="ignore",
    )

    train, test = split_train_test(matrix, test_month)
    train = train[train.date_block_num >= keep_from_month]
    X_train, y_train = xysplit(train)
    X_test, y_test = xysplit(test)
    return (X_train, y_train, X_test, y_test)

In [ ]:
def set_categorical_columns(matrix):
    basic = [
        "shop_id",
        "item_category_id",
        "city_code",
        "month",
        "interaction_month_digital",
        "interaction_month_item_category_id",
    ]
    basic = [c for c in basic if c in matrix.columns]
    matrix.loc[:, basic] = matrix.loc[:, basic].astype("category")
    return matrix

In [ ]:
def censor_lag_features(matrix, candidate_features, replacement_val = 9999):
    # replace any lag features which are invalid due to the shop / item age being less than the lag
    def item_lag_feats(lag):
        return [
            f
            for f in candidate_features
            if f"lag_{lag}" in f
            and "item" in f
            and f[:4] != "shop"
            and "category" not in f
            and "city" not in f
            and "minus" not in f
            and "plus" not in f
            and "sim" not in f
        ]

    def shop_lag_feats(lag):
        return [
            f
            for f in candidate_features
            if f"lag_{lag}" in f
            and "shop" in f
            and "category" not in f
            and "city" not in f
            and "minus" not in f
            and "plus" not in f
            and "sim" not in f
        ]

    lags = range(1,13)
    for lag in lags:
        lag_feats = shop_lag_feats(lag)
        matrix.loc[matrix.shop_age < lag, lag_feats] = replacement_val
        lag_feats = item_lag_feats(lag)
        matrix.loc[matrix.item_age < lag, lag_feats] = replacement_val
    return matrix

In [ ]:
import optuna
from sklearn.metrics import mean_squared_error

import lightgbm as lgbm

class Objective(object):
    def __init__(self, matrix):
        matrix = censor_lag_features(matrix, matrix.columns, replacement_val=9999)
        self.X_train, self.y_train, self.X_valid, self.y_valid = train_test_x_y(
            matrix, test_month=33, keep_from_month=2,
            )

    def __call__(self, trial):
        params = {
            "boosting_type": "gbdt",
            "device_type": "cpu",
            "n_jobs": 11,
            "silent": True,
            "n_estimators": 1000,
            "learning_rate": 0.1,
            "bagging_seed": 3,
            "subsample_for_bin": 300000,
            "max_depth": -1,
            "min_data_in_bin": 1,
            "num_leaves": trial.suggest_int("num_leaves", 255, 2048, log=True),
            "cat_smooth": trial.suggest_float("cat_smooth", 10, 100, log=True),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 10),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.001, 2, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "subsample_freq": trial.suggest_int("subsample_freq", 1, 8),
            "max_bin": trial.suggest_int("max_bin", 16, 255, log=True),
        }

        categoricals = [
            "shop_id",
            "item_category_id",
            "city_code",
            "month",
            "interaction_month_digital",
            "interaction_month_item_category_id",
        ]
        
        categoricals = [c for c in categoricals if c in self.X_train.columns]
        
        early_stopping_rounds=int(1/params['learning_rate'])
        
        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")

        booster = lgbm.LGBMRegressor(**params)

        booster.fit(
            self.X_train,
            self.y_train,
            eval_set=[(self.X_valid, self.y_valid)],
            eval_metric=["rmse"],
            callbacks=[pruning_callback],
            verbose=False,
            categorical_feature=categoricals,
            early_stopping_rounds=early_stopping_rounds
            
        )

        predictions = booster.predict(self.X_valid)
        rmse = mean_squared_error(self.y_valid, predictions.clip(0,20), squared=False)

        return rmse


import warnings

# warnings.filterwarnings("ignore", message="", module="lightgbm", lineno=1286)
warnings.filterwarnings("ignore", message="", module="lightgbm")

study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=15), direction="minimize")
study.optimize(Objective(matrix), n_trials=100, gc_after_trial=True)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
import gc
gc.collect()

In [ ]:
import joblib
joblib.dump(study, "optuna_results_21.03.2021.pk1")

In [ ]:
print("finished everything!")